In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.float_format = '{:,.2f}'.format

Добашнее задание состоит из двух частей  
Задание 1 поможет вам понять, как рассчитывается эффективность работы каналов привлечения трафика.  
Также вы разберётесь, чем реклама в нескольких каналах отличается от рекламы на единственной площадке.

Что нужно сделать
1. Посчитайте эффективность каналов трафика. Для этого рассчитайте ROI для каждого канала. ROI — это отношение всех затрат на канал ко всему доходу, полученному из него.
2. Отсортируйте каналы по убыванию ROI и изобразите их на гистограмме.
3. Ответьте на вопросы:

— Какой канал принёс больше всего денег?  
— Какой канал самый эффективный?   
— В пункте 1 и 2 речь об одном и том же канале или о разных? Если разные это разные каналы, укажите, почему так может быть?  
— Что может произойти, если вложить все деньги в один самый эффективный канал?  

Задание 2 поможет понять, как распределять прибыль между каналами в зависимости от модели атрибуции прибыли  

Решением задания 2 считаются ответы на следующие вопросы:
- Сколько прибыли получим от использования линейной модели?
- Сколько прибыли получим от использования модели time-decay?
- За счет каких каналов линейная модель будет приносить больше выручки, а за счет
каких меньше?
- За счет каких каналов модель time-decay будет приносить больше выручки, а за
счет каких меньше?
- Как вы думаете, почему линейная модель и модель time-decay показали такие
разные результаты?

In [2]:
df = pd.read_excel('tasks.xlsx'
                   , sheet_name='initial_data')
df.head()

,userId,date,trafficSource,cost,value
0,user_9996,2020-01-10,facebook / video,10.25,157.50
1,user_9995,2020-01-03,telegram / posts,15.75,0.00
2,user_9995,2020-01-19,yandex / cpc,8.50,0.00
3,user_9992,2020-01-02,google / cpc,8.25,0.00
4,user_999,2020-01-01,facebook / video,10.25,0.00


In [129]:
df.value.sum()

131345.0

In [3]:
df_source = df.groupby('trafficSource')[['cost', 'value']].sum()
df_source

,cost,value
trafficSource,,
facebook / video,28208.00,33735.0
google / cpc,25137.75,42557.5
telegram / posts,35374.50,29330.0
vk / display,4498.75,4345.0
yandex / cpc,14025.00,21377.5


In [9]:
df_source['ROI'] = round(df_source.value / df_source.cost, 2)
df_source.sort_values('ROI', ascending=False)

,cost,value,ROI
trafficSource,,,
google / cpc,25137.75,42557.5,1.69
yandex / cpc,14025.00,21377.5,1.52
facebook / video,28208.00,33735.0,1.20
vk / display,4498.75,4345.0,0.97
telegram / posts,35374.50,29330.0,0.83


Посмотрим сколько бы было ROI гугла,  
если бы первый раз пришли на наш сайт с него.  
Остальные не будем учитывать

In [133]:
df = df.sort_values('date')

In [126]:
df.head()

,userId,date,trafficSource,cost,value
4999,user_5472,2020-01-01,yandex / cpc,8.50,0.0
2149,user_8022,2020-01-01,facebook / video,10.25,0.0
2150,user_8022,2020-01-01,telegram / posts,15.75,0.0
2151,user_8022,2020-01-01,telegram / posts,15.75,0.0
2152,user_8021,2020-01-01,google / cpc,8.25,0.0


In [3]:
purchase = df[df.value > 0].groupby('userId')[['date']].agg(purchase_date=('date', 'max'))
purchase.head()

,purchase_date
userId,
user_10,2020-01-03
user_1008,2020-01-07
user_1011,2020-01-07
user_1016,2020-01-13
user_1022,2020-01-01


In [4]:
df = df.merge(purchase, on='userId', how='left')
df.head()

,userId,date,trafficSource,cost,value,purchase_date
0,user_9996,2020-01-10,facebook / video,10.25,157.50,2020-01-10
1,user_9995,2020-01-03,telegram / posts,15.75,0.00,NaT
2,user_9995,2020-01-19,yandex / cpc,8.50,0.00,NaT
3,user_9992,2020-01-02,google / cpc,8.25,0.00,NaT
4,user_999,2020-01-01,facebook / video,10.25,0.00,NaT


In [5]:
def func(col):
    if col.name == 'purchase_date':
        return col.iloc[0]
    return col.tolist()
df_temp = df.groupby('userId')[['date', 'trafficSource', 'value', 'cost', 'purchase_date']].agg(func)
df_temp.head()

,date,trafficSource,value,cost,purchase_date
userId,,,,,
user_10,[2020-01-03 00:00:00],[telegram / posts],[215.0],[15.75],2020-01-03
user_10000,"[2020-01-15 00:00:00, 2020-01-17 00:00:00]","[yandex / cpc, google / cpc]","[0.0, 0.0]","[8.5, 8.25]",NaT
user_1002,[2020-01-01 00:00:00],[telegram / posts],[0.0],[15.75],NaT
user_1003,"[2020-01-06 00:00:00, 2020-01-08 00:00:00]","[google / cpc, yandex / cpc]","[0.0, 0.0]","[8.25, 8.5]",NaT
user_1004,"[2020-01-03 00:00:00, 2020-01-12 00:00:00]","[telegram / posts, facebook / video]","[0.0, 0.0]","[15.75, 10.25]",NaT


In [6]:
def func(row):
    value = sum(row['value'])
    if value > 0:
        len_ = len(row['value'])
        row['value_first'] = [value if i == 0 else 0 for i in range(len_)]
        row['value_last'] = row['value']
        row['value_linear'] = [value / len_ for i in range(len_)]
        time_decay = np.zeros(len_)
        for i in range(len_):
            days_to_purchase = (row.purchase_date - row.date[i]).days
            coeff = 2 ** (-days_to_purchase / len_)
            time_decay[i] = coeff
        row['time_decay'] = (time_decay / time_decay.sum()) * value
        return row
    else:
        len_ = len(row['value'])
        temp = [0 for i in range(len_)]
        row['value_first'] = temp
        row['value_last'] = temp
        row['value_linear'] = temp
        row['time_decay'] = temp
        return row    
df_users = df_temp.apply(func, axis=1)
df_users.head()

,date,trafficSource,value,cost,purchase_date,value_first,value_last,value_linear,time_decay
userId,,,,,,,,,
user_10,[2020-01-03 00:00:00],[telegram / posts],[215.0],[15.75],2020-01-03,[215.0],[215.0],[215.0],[215.0]
user_10000,"[2020-01-15 00:00:00, 2020-01-17 00:00:00]","[yandex / cpc, google / cpc]","[0.0, 0.0]","[8.5, 8.25]",NaT,"[0, 0]","[0, 0]","[0, 0]","[0, 0]"
user_1002,[2020-01-01 00:00:00],[telegram / posts],[0.0],[15.75],NaT,[0],[0],[0],[0]
user_1003,"[2020-01-06 00:00:00, 2020-01-08 00:00:00]","[google / cpc, yandex / cpc]","[0.0, 0.0]","[8.25, 8.5]",NaT,"[0, 0]","[0, 0]","[0, 0]","[0, 0]"
user_1004,"[2020-01-03 00:00:00, 2020-01-12 00:00:00]","[telegram / posts, facebook / video]","[0.0, 0.0]","[15.75, 10.25]",NaT,"[0, 0]","[0, 0]","[0, 0]","[0, 0]"


In [7]:
def func(row):
    res = []
    for index, val in row.items():
        index = [(index, i) for i in range(len(val))]
        res.append(pd.Series(val, index=index))
    return pd.concat(res)
res_df = df_users.set_index('purchase_date', append=True).apply(func, axis=1).stack().reset_index(-1, drop=True)
res_df.head()

cost       date  time_decay     trafficSource  \
userId     purchase_date                                                  
user_10    2020-01-03    15.75 2020-01-03      215.00  telegram / posts   
user_10000 NaT            8.50 2020-01-15        0.00      yandex / cpc   
           NaT            8.25 2020-01-17        0.00      google / cpc   
user_1002  NaT           15.75 2020-01-01        0.00  telegram / posts   
user_1003  NaT            8.25 2020-01-06        0.00      google / cpc   

                          value  value_first  value_last  value_linear  
userId     purchase_date                                                
user_10    2020-01-03    215.00       215.00      215.00        215.00  
user_10000 NaT             0.00         0.00        0.00          0.00  
           NaT             0.00         0.00        0.00          0.00  
user_1002  NaT             0.00         0.00        0.00          0.00  
user_1003  NaT             0.00         0.00        0.00          0.00

In [8]:
res_df[res_df.columns.difference(['cost', 'date', 'time_decay', 'trafficSource'])].sum().to_frame()

,0
value,"131,345.00"
value_first,"131,345.00"
value_last,"131,345.00"
value_linear,"131,345.00"


In [9]:
def func(row):
    cost = row.cost
    value = row.value
    row = round(row * 100 / cost, 2)
    row['cost'] = cost
    row['value'] = value
    return row
df_source = res_df.groupby('trafficSource')[['cost', 'value', 'value_last', 'value_first', 'value_linear', 'time_decay']].sum()
df_source.columns = ['cost', 'value', 'last_ROI', 'first_ROI', 'linear_ROI', 'time_decay_ROI']
df_source = df_source.apply(func, axis=1)
df_source

,cost,value,last_ROI,first_ROI,linear_ROI,time_decay_ROI
trafficSource,,,,,,
facebook / video,"28,208.00","33,735.00",119.59,127.26,123.63,121.31
google / cpc,"25,137.75","42,557.50",169.30,162.33,170.27,178.52
telegram / posts,"35,374.50","29,330.00",82.91,76.16,77.76,78.66
vk / display,"4,498.75","4,345.00",96.58,103.42,90.04,77.69
yandex / cpc,"14,025.00","21,377.50",152.42,164.35,157.66,149.25


- Сколько прибыли получим от использования линейной модели?
- Сколько прибыли получим от использования модели time-decay?
- За счет каких каналов линейная модель будет приносить больше выручки, а за счет
каких меньше?
- За счет каких каналов модель time-decay будет приносить больше выручки, а за
счет каких меньше?
- Как вы думаете, почему линейная модель и модель time-decay показали такие
разные результаты?

In [10]:
new_budget = 1e6
sum_cost = df_source.cost.sum()
df_source['new_budget'] = round(new_budget * df_source.cost / sum_cost, 2)
df_source

,cost,value,last_ROI,first_ROI,linear_ROI,time_decay_ROI,new_budget
trafficSource,,,,,,,
facebook / video,"28,208.00","33,735.00",119.59,127.26,123.63,121.31,"263,026.37"
google / cpc,"25,137.75","42,557.50",169.30,162.33,170.27,178.52,"234,397.73"
telegram / posts,"35,374.50","29,330.00",82.91,76.16,77.76,78.66,"329,850.62"
vk / display,"4,498.75","4,345.00",96.58,103.42,90.04,77.69,"41,948.73"
yandex / cpc,"14,025.00","21,377.50",152.42,164.35,157.66,149.25,"130,776.55"


In [25]:
def func(row):
    res = pd.Series([], dtype=float)
    delta = (row.first_ROI - row.last_ROI) / 100
    res['old_spend'] = row.new_budget
    new_spend = row.new_budget * (1 + delta)
    res['new_spend'] = new_spend
    old_rvenue = row.last_ROI * new_spend / 100
    res['old_revenue'] = old_rvenue
    new_revenue = row.first_ROI * new_spend / 100
    res['new_revenue'] = new_revenue
    res['delta_revenue'] = new_revenue - old_rvenue
    return res
df_first = df_source.apply(func, axis=1)
df_first.loc['Total'] = df_first.sum()
df_first.style.background_gradient(cmap='RdYlGn', subset='delta_revenue').format(precision=2)

,old_spend,new_spend,old_revenue,new_revenue,delta_revenue
trafficSource,,,,,
facebook / video,263026.37,283200.49,338679.47,360400.95,21721.48
google / cpc,234397.73,218060.21,369175.93,353977.14,-15198.80
telegram / posts,329850.62,307585.70,255019.31,234257.27,-20762.03
vk / display,41948.73,44818.02,43285.25,46350.80,3065.55
yandex / cpc,130776.55,146378.19,223109.64,240572.56,17462.92
Total,1000000.00,1000042.62,1229269.60,1235558.71,6289.12


In [26]:
def func(row):
    res = pd.Series([], dtype=float)
    delta = (row.linear_ROI - row.last_ROI) / 100
    res['old_spend'] = row.new_budget
    new_spend = row.new_budget * (1 + delta)
    res['new_spend'] = round(new_spend, 2)
    old_rvenue = row.last_ROI * new_spend / 100
    res['old_revenue'] = round(old_rvenue, 2)
    new_revenue = row.linear_ROI * new_spend / 100
    res['new_revenue'] = round(new_revenue, 2)
    res['delta_revenue'] = round(new_revenue - old_rvenue, 2)
    return res
df_linear = df_source.apply(func, axis=1)
df_linear.loc['Total'] = df_linear.sum()
df_linear.style.background_gradient(cmap='RdYlGn', subset='delta_revenue').format(precision=2)

,old_spend,new_spend,old_revenue,new_revenue,delta_revenue
trafficSource,,,,,
facebook / video,263026.37,273652.64,327261.19,338316.75,11055.57
google / cpc,234397.73,236671.39,400684.66,402980.37,2295.71
telegram / posts,329850.62,312863.31,259394.97,243282.51,-16112.46
vk / display,41948.73,39205.28,37864.46,35300.44,-2564.03
yandex / cpc,130776.55,137629.24,209774.49,216986.26,7211.77
Total,1000000.00,1000021.86,1234979.77,1236866.33,1886.56


In [27]:
def func(row):
    res = pd.Series([], dtype=float)
    delta = (row.time_decay_ROI - row.last_ROI) / 100
    res['old_spend'] = row.new_budget
    new_spend = row.new_budget * (1 + delta)
    res['new_spend'] = round(new_spend, 2)
    old_rvenue = row.last_ROI * new_spend / 100
    res['old_revenue'] = round(old_rvenue, 2)
    new_revenue = row.time_decay_ROI * new_spend / 100
    res['new_revenue'] = round(new_revenue, 2)
    res['delta_revenue'] = round(new_revenue - old_rvenue, 2)
    return res
df_time_decay = df_source.apply(func, axis=1)
df_time_decay.loc['Total'] = df_time_decay.sum()
df_time_decay.style.background_gradient(cmap='RdYlGn', subset='delta_revenue').format(precision=2)

,old_spend,new_spend,old_revenue,new_revenue,delta_revenue
trafficSource,,,,,
facebook / video,263026.37,267550.42,319963.55,324565.42,4601.87
google / cpc,234397.73,256009.20,433423.58,457027.63,23604.05
telegram / posts,329850.62,315831.97,261856.29,248433.43,-13422.86
vk / display,41948.73,34024.61,32860.97,26433.72,-6427.25
yandex / cpc,130776.55,126630.93,193010.87,188996.67,-4014.20
Total,1000000.00,1000047.13,1241115.26,1245456.87,4341.61
